In [ ]:
!pip install unsloth unsloth_zoo 'peft==0.15.2' 'transformers==4.52.4'
!pip install --pre --upgrade --no-cache-dir torchvision --extra-index-url https://download.pytorch.org/whl/nightly/cu128

In [2]:
import transformers
transformers.__version__

'4.52.4'

In [15]:
from unsloth import FastVisionModel # FastLanguageModel for LLMs
import torch

model, tokenizer = FastVisionModel.from_pretrained(
    "unsloth/Qwen2.5-VL-7B-Instruct-bnb-4bit",
    # "Cherran/grpo_ckpt_75_bs_4_VL_medical_qwen2point5_7b",
    load_in_4bit = True, # Use 4bit to reduce memory use. False for 16bit LoRA.
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for long context
)

==((====))==  Unsloth 2025.6.12: Fast Qwen2_5_Vl patching. Transformers: 4.52.4.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.643 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [16]:
from datasets import load_dataset

dataset = load_dataset("Cherran/train_orig_medical_summaries", split = 'train')
dataset

Dataset({
    features: ['image', 'prompt', 'summary'],
    num_rows: 732
})

In [17]:

def convert_to_conversation(sample):
    """
    Converts a sample with a list of images into the Unsloth VLM conversation format.
    """
    user_content = [
        {"type": "text", "text": sample['prompt']}
    ]

    for image_object in sample["image"]:
        user_content.append(
            {"type": "image", "image": image_object}
        )

    assistant_response = sample['summary']

    conversation = [
        {
            "role": "user",
            "content": user_content  # Use the dynamically built list here
        },
        {
            "role": "assistant",
            "content": [
                {"type": "text", "text": assistant_response}
            ]
        },
    ]
    return {"messages": conversation}

In [ ]:
dataset = dataset.shuffle(seed = 42)

In [ ]:
converted_dataset = [convert_to_conversation(sample) for sample in dataset]

In [20]:
converted_dataset[0]

{'messages': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'You are an excellent medical paragraph generator. I have drug names and Adverse Drug reactions Keywords. Consolidate all information in one paragrah without adding any extra knowledge or text.\n  drug_names: herceptin\n  physical_adverse_effects: joint pain, knee pain, hip pain, back pain\n\n  Strcitly output the paragraph no explanations.\n'}]},
  {'role': 'assistant',
   'content': [{'type': 'text',
     'text': 'Herceptin may cause several physical adverse effects including joint pain, knee pain, hip pain, and back pain.'}]}]}

In [21]:
model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers     = True, # False if not finetuning vision layers
    finetune_language_layers   = True, # False if not finetuning language layers
    finetune_attention_modules = True, # False if not finetuning attention layers
    finetune_mlp_modules       = True, # False if not finetuning MLP layers

    r = 32,
    lora_alpha = 32,
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None, # And LoftQ
    # target_modules = "all-linear", # Optional now! Can specify a list if needed
)

### This is optional, i added dummy image in places where images is not present

In [ ]:
from PIL import Image

dummy_image = Image.new('RGB', (512, 512), color = 'black')

def add_dummy_image(example):
    if not example["image"]:  
        example["image"] = [dummy_image] 
    return example

dataset_with_dummies = dataset.map(add_dummy_image)

In [28]:
converted_dataset = [convert_to_conversation(sample) for sample in dataset_with_dummies]
converted_dataset[0]

{'messages': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'You are an excellent medical paragraph generator. I have drug names and Adverse Drug reactions Keywords. Consolidate all information in one paragrah without adding any extra knowledge or text.\n  drug_names: herceptin\n  physical_adverse_effects: joint pain, knee pain, hip pain, back pain\n\n  Strcitly output the paragraph no explanations.\n'},
    {'type': 'image',
     'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=512x512>}]},
  {'role': 'assistant',
   'content': [{'type': 'text',
     'text': 'Herceptin may cause several physical adverse effects including joint pain, knee pain, hip pain, and back pain.'}]}]}

#### traning

In [ ]:
from unsloth.trainer import UnslothVisionDataCollator
from trl import SFTTrainer, SFTConfig

FastVisionModel.for_training(model) # Enable for training!

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    data_collator = UnslothVisionDataCollator(model, tokenizer), # Must use!
    train_dataset = converted_dataset,
    args = SFTConfig(
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 2,
        warmup_steps = 5,
        # max_steps = 30,
        num_train_epochs = 1, # Set this instead of max_steps for full training runs
        learning_rate = 5e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",     # you can use Weights and Biases

        # You MUST put the below items for vision finetuning:
        remove_unused_columns = False,
        dataset_text_field = "",
        dataset_kwargs = {"skip_prepare_dataset": True},
        dataset_num_proc = 4,
        max_seq_length = 1024,
    ),
)

Unsloth: Model does not have a default image size - using 512


In [30]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 732 | Num Epochs = 1 | Total steps = 46
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 2 x 1) = 16
 "-____-"     Trainable parameters = 103,043,072 of 7,000,000,000 (1.47% trained)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,8.057000
2,7.873800
3,7.150800
4,4.207000
5,3.082700
6,2.749000
7,2.038200
8,1.532800
9,1.081500
10,0.768600


In [ ]:
from huggingface_hub import HfApi

model_path = "outputs/checkpoint-46"
username = "Cherran"
MODEL_NAME = "VL_lr_5e4_medical_qwen2point5_7b"
api = HfApi(token=  "HF_key")

api.create_repo(
    repo_id = f"{username}/{MODEL_NAME}",
    repo_type="model"
)

api.upload_folder(
    repo_id = f"{username}/{MODEL_NAME}",
    folder_path = model_path
)

  0%|          | 0/6 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

optimizer.pt:   0%|          | 0.00/164M [00:00<?, ?B/s]

rng_state.pth:   0%|          | 0.00/14.2k [00:00<?, ?B/s]

scheduler.pt:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.62k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Cherran/VL_medical_qwen2point5_7b/commit/2b0f0d27596f15207d80e31f94bac5e5a40e51f5', commit_message='Upload folder using huggingface_hub', commit_description='', oid='2b0f0d27596f15207d80e31f94bac5e5a40e51f5', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Cherran/VL_medical_qwen2point5_7b', endpoint='https://huggingface.co', repo_type='model', repo_id='Cherran/VL_medical_qwen2point5_7b'), pr_revision=None, pr_num=None)

In [56]:
input_prompt = "You are an excellent medical paragraph generator. I have drug names and Adverse Drug reactions Keywords. Consolidate all information in one paragrah without adding any extra knowledge or text.\n  drug_names: Taxol\n  physical_adverse_effects: Pain in left hand/thumb, port protruding through skin, brace on hand\n\n  Strcitly output the paragraph no explanations.\n"

In [57]:
FastVisionModel.for_inference(model) # Enable for inference!

messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": input_prompt, }
    ]}
]
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt = True)
inputs = tokenizer(
    text = input_text,
    add_special_tokens = False,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1 , min_p = 0.1)

Taxol has been associated with several adverse effects, including pain in the left hand/thumb, a port that protrudes through the skin, and a brace on the hand. These symptoms may indicate a range of issues related to its use or administration.<|im_end|>


### validation

In [10]:
import pandas as pd
df_val = pd.read_csv('val_VL_medical_qwen_zs_sft.csv')
df_val.head(1)

,Unnamed: 0,Links,Posts,Preprocessed Posts,drug_names,adverse_effects,severity,physical_adverse_effects,non_physical_adverse_effects,side_harmful_effects,image_description_with_phyadv,phy_summary,non_phy_summary,VL_qwen_phy_sum_sft,VL_qwen_non_phy_sum_sft,VL_qwen_phy_sum_zs,VL_qwen_non_phy_sum_zs
0,0,https://csn.cancer.org/discussion/comment/1239...,Xeloda on the Web.. Awesome website\n I switc...,xeloda on the web awesome website i switched o...,Xeloda,Yes,Mild,HandFoot syndrome,NaN,NaN,The image shows a hand with a visible silver b...,Xeloda is associated with adverse drug reactio...,NaN,Xeloda (5-FU) is associated with potential adv...,NaN,Xeloda is associated with a significant advers...,NaN


In [2]:
from datasets import load_dataset

dataset = load_dataset("Cherran/draft_medical_summaries", split = 'train')
dataset

Dataset({
    features: ['image', 'phy_prompt', 'non_phy_prompt', 'phy_summary', 'non_phy_summary'],
    num_rows: 300
})

In [ ]:
def generate_response(sample, prompt_type='phy_prompt', temperature=1.0):

    # start with the text prompt
    user_content = [
        {"type": "text", "text": sample[prompt_type]}
    ]

    images = sample.get("image", [])
    if images and not isinstance(images, list):
        images = [images]

    for image_object in images:
        user_content.append(
            {"type": "image"} 
        )
        
    conversation = [
        {
            "role": "user",
            "content": user_content  
        }
    ]
    
    # apply the chat template
    final_prompt_string = tokenizer.apply_chat_template(
        conversation, 
        tokenize=False, 
        add_generation_prompt=True
    )
    
    if images:
        print("Processing with text and images...")
        inputs = tokenizer(
            text=final_prompt_string,
            images=images,
            return_tensors="pt"
        ).to("cuda")
    else:

        print("Processing with text only...")
        inputs = tokenizer(
            text=final_prompt_string,
            return_tensors="pt"
        ).to("cuda")

    output_ids = model.generate(**inputs, max_new_tokens=128, use_cache=True, temperature=temperature, min_p=0.1)
    
    input_ids_length = inputs["input_ids"].shape[1]
    newly_generated_ids = output_ids[:, input_ids_length:]
    
    response_text = tokenizer.batch_decode(newly_generated_ids, skip_special_tokens=True)[0]
    
    return response_text.strip()

In [ ]:
# FastVisionModel.for_inference(model) # not required

4


In [ ]:
from tqdm import tqdm 

df_val['VL_qwen_phy_sum_grpo_ckpt_75_bs_4'] = ''


for index in tqdm(range(0, len(dataset))):
    sample = dataset[index]
    if sample['phy_prompt'] == '' :
        continue   
    response = generate_response(sample, prompt_type = 'phy_prompt', temperature = 1.0)
    print(response)
    df_val.at[index , 'VL_qwen_phy_sum_grpo_ckpt_75_bs_4'] = response

  0%|          | 0/300 [00:00<?, ?it/s]

Processing with text only...


  0%|          | 1/300 [00:12<1:00:50, 12.21s/it]

The adverse drug reaction associated with Xeloda is Hand Foot syndrome.
Processing with text only...


  1%|          | 2/300 [00:18<43:40,  8.79s/it]  

...

The adverse reaction associated with taxol is leg pain, numb fingers, and numb toes.
Processing with text only...


100%|█████████▉| 299/300 [19:09<00:02,  2.97s/it]

The adverse reaction associated with Co-codamol is discomfort in the lower abdomen.
Processing with text only...


100%|██████████| 300/300 [19:15<00:00,  3.85s/it]

The adverse drug reactions associated with Cape and morphine include pain in the scar area, fatigue, dry mouth, indigestion, constipation, muscle weakness in legs, and dizziness.


In [38]:
df_val['VL_qwen_non_phy_sum_zs'] = ''
for index in tqdm(range(0, len(dataset))):
    sample = dataset[index]
    if sample['non_phy_prompt'] == '' :
        continue   
    response = generate_response(sample, prompt_type = 'non_phy_prompt')
    print(response)
    df_val.at[index , 'VL_qwen_non_phy_sum_zs'] = response

  0%|          | 0/300 [00:00<?, ?it/s]

Processing with text only...


  0%|          | 1/300 [00:04<22:25,  4.50s/it]

Xeloda is associated with a significant adverse drug reaction known as Hand Foot Syndrome, which can manifest as symptoms affecting the skin of the hands and feet, potentially impacting patients' quality of life.
Processing with text only...


  1%|          | 2/300 [00:09<23:02,  4.64s/it]

...

Letrozole, Arimidex, Exemestane, Tamoxifen, and Femara are medications commonly used to treat breast cancer. These drugs may cause various side effects, including insomnia, which is a common non-physical adverse reaction associated with their use.
Processing with text only...


 99%|█████████▉| 297/300 [06:46<00:02,  1.23it/s]

Carb and Taxol may cause patients to feel self-conscious, very tired, or exhausted as side effects of their use.
Processing with text only...


100%|██████████| 300/300 [06:49<00:00,  1.36s/it]

Cape and morphine can cause non-physical adverse effects such as mental confusion, making patients feel "not completely with it."


In [40]:
sum(df_val['VL_qwen_phy_sum_grpo'] == '') , sum(df_val['VL_qwen_non_phy_sum_grpo'] == '')

(39, 176)

In [13]:
sum(df_val['VL_qwen_phy_sum_grpo_ckpt_75_bs_4'] == '')

39

In [41]:
df_val.columns

Index(['Unnamed: 0', 'Links', 'Posts', 'Preprocessed Posts', 'drug_names',
       'adverse_effects', 'severity', 'physical_adverse_effects',
       'non_physical_adverse_effects', 'side_harmful_effects',
       'image_description_with_phyadv', 'phy_summary', 'non_phy_summary',
       'VL_qwen_phy_sum_sft', 'VL_qwen_non_phy_sum_sft', 'VL_qwen_phy_sum_zs',
       'VL_qwen_non_phy_sum_zs'],
      dtype='object')

In [14]:
df_val.to_csv('val_VL_medical_qwen_draft_grpo.csv' , index = False)